In [1]:
import numpy as np
import random

In [10]:
class stateClass:
    def __init__(self,state, K, reward = -1):
        self.K = K
        self.Trap_y1 = [0,1,2,3,4,5,6,7]
        self.Trap_x1 = [4]
        self.Trap_y2 = [5,6,7,8,9,10,11,12]
        self.Trap_x2 = [8]
        self.isStart = self.isStart(state)
        self.isEnd = self.isTerminal(state)
        self.state = state
        self.reward = reward
    
        if (self.state == (self.K-1,self.K-1)):
            self.reward = 2*(self.K-1)
        elif ((self.state[0] in self.Trap_x1 and self.state[1] in self.Trap_y1) or (self.state[0] in self.Trap_x2 and self.state[1] in self.Trap_y2)):
            self.reward = -2*(self.K-1)
            
    def isTerminal(self, state):
        if (state==(self.K-1,self.K-1)):
            return True
        return False
    
    def isStart(self, state):
        if (state==(0,0)):
            return True
        return False
      
    def getStates(self):
        return (self.state[0],self.state[1])


    def getReward(self):
        if self.state == (self.K-1,self.K-1):
            return 2*(self.K-1)

        else:
            return -1

# greedy policy

In [11]:
class Grid():
    def __init__(self,K):
        self.K = K
        self.action_value_state = np.zeros((self.K,self.K))
        self.policy = [["" for i in range(self.K)] for i in range(self.K)]
        
    def q(self, alpha=0.01, num_episodes=1500, gamma=0.9):
        for ep in range(num_episodes):
            init = [i for i in range(self.K)]
            m = random.choice(init)
            n = random.choice(init)
            state=stateClass((m,n),self.K)
            if ((state.getStates()[0] in state.Trap_x1 and state.getStates()[1] in state.Trap_y1) or (state.getStates()[0] in state.Trap_x2 and state.getStates()[1] in state.Trap_y2)):
                continue
            while True:
                action = self.take_greedy_action(state.getStates())
                next_state = self.step(state.getStates(),action)

                next_state_ = stateClass(next_state,self.K)
                reward = next_state_.getReward()

                self.action_value_state[state.getStates()[0]][state.getStates()[1]] += alpha*(reward+gamma*self.action_value_state[next_state[0]][next_state[1]])
                if ((next_state[0] in state.Trap_x1 and next_state[1] in state.Trap_y1) or (next_state[0] in state.Trap_x2 and next_state[1] in state.Trap_y2)):
                    break
                self.policy[state.getStates()[0]][state.getStates()[1]] = action
                state=next_state_
                if action == 'T':
                    break

        return self.action_value_state,self.policy

    def take_greedy_action(self,state, gamma=0.9):
        dct={}
        possible_actions = self.getPossibleActions(state)
        for i, val in enumerate(possible_actions):
            if val == ['T']:
                continue
            next_state = self.step(state,val)
            v_next = self.action_value_state[next_state[0]][next_state[1]]
            next_state = stateClass((next_state[0],next_state[1]),self.K)
            reward = next_state.getReward()
            v_new = reward + gamma*v_next
            dct[val]=v_new

        valist = [val for val in dct.values()]
        max_value = max(valist)
        self.action_value_state[state[0]][state[1]] = max_value
        best_action = self.get_best_action(dct,max_value)
        return best_action


    def step(self, state,action):
        i = state[0]
        j = state[1]
        if action == 'S': # SOUTH
            if i == self.K-1:
                return None
            else:
                i+=1        
          
        elif action == 'E':#EAST
            if j == self.K-1:
                return None
            else:
                j+=1
                        
        elif action == 'N':#NORTH
            if i == 0:
                return None
            else:
                i-=1
                                      
        elif action == 'W':#WEST
            if j == 0:
                return None
            else:
                j-=1
            
        return (i,j)
    
    def getPossibleActions(self, state):
        
        if state[0]==0:
            if state[1]==0:
                return ['E','S']
            elif state[1]==self.K-1:
                return ['S','W']
            else:
                return ['W','E','S']

        elif state[0]==self.K - 1:
            if state[1]==0:
                return ['N','E']
            elif state[1]==self.K-1:
                return ['T']
            
            else:
                return ['N','E','W']

        elif state[1]==0:
            return ['S','N','E']
        
        elif state[1]==self.K-1:
            return ['N','S','W']

        else:
            return ['N','E','S','W']
        
    def get_best_action(self,d,val):
        for key, value in d.items():
#             print("the dict",d)
            if val == value:
#                 print("the action chosen",key)
                return key
            

In [12]:
k = int(input("Enter k the size of the gridWorld: "))

Enter k the size of the gridWorld: 12


In [13]:
grid = Grid(k)


In [14]:
action_value_state,policy = grid.q(num_episodes = 2000)

In [15]:
policy

[['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'S', 'W', 'S', 'S'],
 ['S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'W'],
 ['S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'S'],
 ['', '', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S'],
 ['', '', '', '', '', '', '', '', 'E', 'S', 'S', 'S'],
 ['S', 'S', 'E', 'E', 'S', 'E', 'S', 'S', 'S', 'S', 'S', 'S'],
 ['S', 'E', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'W'],
 ['S', 'E', 'E', 'E', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'N'],
 ['S', 'E', 'E', 'E', 'S', '', '', '', '', '', '', ''],
 ['S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S'],
 ['S', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S'],
 ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'T']]

In [8]:
def print_policy():
    m = int(input("Entrer la ligne: "))
    n = int(input("Entrer la colonne: "))
    while(m in [4] and n in [1,2,3,4,5,6,7,8] or m in[8] and n in [5,6,7,8,9,10,11,12]):
        m = int(input("Entrer la ligne: "))
        n = int(input("Entrer la colonne: "))
    l = []
    start = (m,n)
    print(f"starting from {start} follow this path")
    l.append(policy[m][n])
    while(policy[m][n]!='T'):
        if policy[m][n]=='E':
            n+=1
            l.append(policy[m][n])
        elif policy[m][n]=='W':
            n-=1
            l.append(policy[m][n])
        elif policy[m][n]=='S':
            m+=1
            l.append(policy[m][n])
        elif policy[m][n]=='N':
            m-=1
            l.append(policy[m][n])
        elif policy[m][n]=="":
            print("you are in the trap")
            break
    return l

In [9]:
road = print_policy()
print(road)

Entrer la ligne: 2
Entrer la colonne: 1
starting from (2, 1) follow this path
['S', 'W', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'T']


# epsilon greedy policy

In [16]:
class Grid_ep():
    def __init__(self,K):
        self.K = K
        self.action_value_state = np.zeros((self.K,self.K))
        self.policy = [["" for i in range(self.K)] for i in range(self.K)]
        
    def q(self, alpha=0.01, num_episodes=1500, gamma=0.9,epsilon=0.1):
        for ep in range(num_episodes):
            init = [i for i in range(self.K)]
            m = random.choice(init)
            n = random.choice(init)
            state=stateClass((m,n),self.K)
            if ((state.getStates()[0] in state.Trap_x1 and state.getStates()[1] in state.Trap_y1) or (state.getStates()[0] in state.Trap_x2 and state.getStates()[1] in state.Trap_y2)):
                continue
            while True:
                action = self.take_epsilon_greedy_action(state.getStates(),gamma,epsilon)
                next_state = self.step(state.getStates(),action)

                next_state_ = stateClass(next_state,self.K)
                reward = next_state_.getReward()

                self.action_value_state[state.getStates()[0]][state.getStates()[1]] += alpha*(reward+gamma*self.action_value_state[next_state[0]][next_state[1]]-self.action_value_state[state.getStates()[0]][state.getStates()[1]])
                if ((next_state[0] in state.Trap_x1 and next_state[1] in state.Trap_y1) or (next_state[0] in state.Trap_x2 and next_state[1] in state.Trap_y2)):
                    break
                self.policy[state.getStates()[0]][state.getStates()[1]] = action
                state=next_state_
                if action == 'T':
                    break

        return self.action_value_state,self.policy

    def take_epsilon_greedy_action(self,state, gamma,epsilon):
        if np.random.random() < epsilon:
            return random.choice(self.getPossibleActions(state))
        else:

            dct={}
            possible_actions = self.getPossibleActions(state)
            for i, val in enumerate(possible_actions):
                if val == ['T']:
                    continue
                next_state = self.step(state,val)
                v_next = self.action_value_state[next_state[0]][next_state[1]]
                next_state = stateClass((next_state[0],next_state[1]),self.K)
                reward = next_state.reward
                v_new = reward + gamma*v_next
                dct[val]=v_new

            valist = [val for val in dct.values()]
            max_value = max(valist)
            self.action_value_state[state[0]][state[1]] = max_value
            best_action = self.get_best_action(dct,max_value)
            return best_action


    def step(self, state,action):
        i = state[0]
        j = state[1]
        if action == 'S': # South
            if i == self.K-1:
                return None
            else:
                i+=1        
          
        elif action == 'E':#EAST
            if j == self.K-1:
                return None
            else:
                j+=1
                        
        elif action == 'N':#North
            if i == 0:
                return None
            else:
                i-=1
                                      
        elif action == 'W':#WEST
            if j == 0:
                return None
            else:
                j-=1
            
        return (i,j)
    
    def getPossibleActions(self, state):
        
        if state[0]==0:
            if state[1]==0:
                return ['E','S']
            elif state[1]==self.K-1:
                return ['S','W']
            else:
                return ['W','E','S']

        elif state[0]==self.K - 1:
            if state[1]==0:
                return ['N','E']
            elif state[1]==self.K-1:
                return ['T']
            
            else:
                return ['N','E','W']

        elif state[1]==0:
            return ['S','N','E']
        
        elif state[1]==self.K-1:
            return ['N','S','W']

        else:
            return ['N','E','S','W']
        
    def get_best_action(self,d,val):
        for key, value in d.items():
#             print("the dict",d)
            if val == value:
#                 print("the action chosen",key)
                return key
            

In [17]:
grid = Grid_ep(k)
action_value_state,policy = grid.q(num_episodes = 4000, gamma = 0.9,epsilon = 0.8)

In [18]:
policy

[['E', 'S', 'S', 'S', 'E', 'E', 'E', 'S', 'E', 'E', 'S', 'S'],
 ['E', 'E', 'E', 'E', 'E', 'E', 'S', 'W', 'N', 'S', 'W', 'S'],
 ['E', 'S', 'E', 'N', 'E', 'E', 'S', 'S', 'N', 'S', 'S', 'W'],
 ['N', 'E', 'E', 'N', 'N', 'E', 'E', 'W', 'S', 'W', 'E', 'S'],
 ['', '', '', '', '', '', '', '', 'S', 'W', 'S', 'W'],
 ['S', 'W', 'S', 'S', 'E', 'S', 'E', 'E', 'E', 'E', 'S', 'N'],
 ['E', 'E', 'E', 'E', 'S', 'W', 'S', 'W', 'W', 'W', 'W', 'W'],
 ['S', 'N', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
 ['S', 'E', 'N', 'S', 'S', '', '', '', '', '', '', ''],
 ['S', 'S', 'E', 'E', 'S', 'W', 'E', 'E', 'S', 'E', 'W', 'S'],
 ['E', 'E', 'S', 'S', 'E', 'E', 'N', 'W', 'S', 'S', 'E', 'S'],
 ['E', 'E', 'E', 'E', 'N', 'N', 'W', 'N', 'W', 'E', 'N', 'T']]

In [19]:
road = print_policy()
print(road)

Entrer la ligne: 0
Entrer la colonne: 0
starting from (0, 0) follow this path


KeyboardInterrupt: 